# Lab 9 Group 21


### New Vuln 1 :Improper Password validation and secure-by-default handling.
SampleNetworkServer.py does not properly handle commands chained to invalid AUTH and Logout requests. An attacker can submit a invalid password to the AUTH command and still be allowed to execute all other commands such as SET_DEGF or UPDATE_TEMP that would impact the legitimate client.
### New Vuln 2: Inaccurate Incubator Temperature Reporting on unit change
Both updateInfTemp and updateIncTemp use the same calculator for temperature regardless of what units the different systems are using causing inconsistencies in the reporting of temperatures.

In [40]:
#Exploit
import unittest
import socket

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

class TestExploits(unittest.TestCase):
    # Vuln 1 Exploit Test
    def test_bad_password(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        input_request=b'AUTH basd;GET_TEMP;'
        s_rl.sendto(input_request, ("127.0.0.1", 23456))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        output_responce=str(output_responce.strip(),"utf8")
        s_rl.close()
        print(output_responce)
        self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid auth.")
    # Vuln 2 Unit change Test
    def test_unit_change(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        request_1=b'AUTH basd;GET_TEMP;'
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        deg_k=float(output_responce.strip())
        print(f"deg kelvin={deg_k}")
        input_request=b'AUTH basd;SET_DEGF'
        s_rl.sendto(input_request, ("127.0.0.1", 23457))
        # output_responce, real_server_addr = s_rl.recvfrom(1024)
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        deg_f=float(output_responce.strip())
        print(f"deg f={deg_f}")
        request_3=b'AUTH basd;SET_DEGC;'
        s_rl.sendto(request_3, ("127.0.0.1", 23457))
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        s_rl.close()
        deg_c=float(output_responce.strip())
        print(f"deg celcius={deg_c}")
        self.assertNotAlmostEqual(deg_c,deg_f,places=0,msg="Reported Temp in F and C should not be the same.")
        self.assertNotAlmostEqual(deg_k,deg_c,places=0,msg="Reported Temp in K and C should not be the same.")
        self.assertNotAlmostEqual(deg_k,deg_f,places=0,msg="Reported Temp in K and F should not be the same.")



res = unittest.main(argv=[''], verbosity=3, exit=False)


test_unit_change (__main__.TestExploits) ... 

deg kelvin=73.805282
deg f=164.86457


FAIL

FAIL: test_unit_change (__main__.TestExploits)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tc/3ljmtg750xz6n4_dy5k9l0mm0000gn/T/ipykernel_63586/761996333.py", line 46, in test_unit_change
    self.assertNotAlmostEqual(deg_k,deg_c,places=0,msg="Reported Temp in K and C should not be the same.")
AssertionError: 73.805282 == 73.82463 within 0 places : Reported Temp in K and C should not be the same.

----------------------------------------------------------------------
Ran 1 test in 7.743s

FAILED (failures=1)


deg celcius=73.82463
